# Testing the implementation

In [2]:
import os
import gzip
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
import tensorflow_datasets as tfds

from tuner import *
from models import *
from trainer import *
from preprocess import *
from callbacks import *

/Users/customer/miniforge3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 0 Data

Obtain the full data

In [3]:
parent_path = str(pathlib.Path(os.getcwd()).parent)
df = pd.read_csv(os.path.join(parent_path, 'data/data.csv'))
df = df.drop(['Unnamed: 0'], axis=1)
df.sample(5)

,optionid,securityid,strike,callput,date_traded,contract_price,market_price,underlyings_price,contract_volume,days_to_maturity,moneyness,rate,volatility
64127,156792096.0,702263.0,14.00,C,2007-07-31,0.0575,0.05000,12.459380,15.0,87.0,0.889956,0.053617,0.185943
19125,150041380.0,506497.0,55.00,C,2006-09-12,0.3600,0.37300,51.259702,32.0,94.0,0.931995,0.054609,0.110531
106345,174523773.0,702263.0,21.85,C,2021-03-29,0.1100,0.04950,21.699851,12.0,4.0,0.993128,0.000826,0.156564
68071,157895599.0,702263.0,11.60,C,2012-04-10,0.2100,0.19625,10.151250,372.0,255.0,0.875108,0.005222,0.258637
99852,165109107.0,702263.0,18.25,C,2020-01-17,0.1200,0.11500,18.260500,3.0,7.0,1.000575,0.015771,0.144167


Create the first neueral network that uses the same set of inputs as the *Black-Scholes model*

In [4]:
dataframe_BS = np.vstack((df['strike'].values,
                      df['underlyings_price'].values,
                      df['days_to_maturity'].values,
                      df['volatility'].values,
                      df['rate'].values,
                      df['contract_price'].values)).T                

In [5]:
train_ds, valid_ds, test_ds = pipeline1(dataframe_BS, scaling=False)

train_copy, valid_copy, test_copy = pipeline1(dataframe_BS, prefetch=False)

(85999, 6) (10750, 6) (10750, 6)
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

(85999, 6) (10750, 6) (10750, 6)


2022-09-09 19:16:27.232451: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-09 19:16:27.232610: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## 1 Start training

Define some parameters

In [6]:
print_num_epochs = 5 # print progress every print_num_epochs epochs

path_to_save = os.path.join(parent_path, 'NeuralNetwork/models/roughwork')  # path to save the model

patience = 10  

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

loss = tf.keras.losses.MeanAbsoluteError(name='loss')

metrics = tf.keras.metrics.MeanAbsolutePercentageError(name='accuracy')

num_epochs = 10  

input_shape = (5,)

num_layers = 3

hidden_units = [14, 14, 14]

output_shape = (1, )

batchnorm = True  

dropout = None

model = getModel(input_shape = input_shape,
                num_layers   = num_layers,
                 hidden_units = hidden_units,
                 output_shape = output_shape,
                 batchnorm = batchnorm,
                 dropout = dropout)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5)]               0         
                                                                 
 flatten (Flatten)           (None, 5)                 0         
                                                                 
 dense (Dense)               (None, 14)                84        
                                                                 
 batch_normalization (BatchN  (None, 14)               56        
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 14)                210       
                                                                 
 batch_normalization_1 (Batc  (None, 14)               56        
 hNormalization)                                             

Define some callbacks  

In [7]:
ckpt = CheckpointCallback(path_to_save)
printing =PrintProgress(num_epochs=print_num_epochs)
early_stop = tf.keras.callbacks.EarlyStopping(patience=patience, monitor='val_loss')
callbacks = [ckpt, printing, early_stop]

In [8]:
testing = False
if testing:
    history = compile_and_fit(model,
                          optimizer,
                          loss,
                          num_epochs,
                          train_ds,
                          valid_ds,
                          metrics,
                          callbacks,
                          verbose=True
                          )

# 1.1 Tuning the hyperparameters

To tune the hyperparameter, we can use `keras_tuner`, which provides `RandomSearch`, `Hyperband`, and `BayesianSearch` methods for searching the best hyperparameters.  

To get started, we only tune the number of layers and the hidden units in each layer first, then we tune with learning rate together.

In [9]:
import keras_tuner

We initialize the `keras_tuner` object for tuning hyperparameters; currently, there are two hyper models implemented, `tuneLayer` and `tuneLR`

In [10]:
# random_tuner = keras_tuner.RandomSearch(
#     hypermodel=tunedModel, # the hypermodel to tune # can be tuneLR or tuneLayer
#     objective="val_loss", # the objective to optimize
#     max_trials=3, # the maximum number of trials to run
#     executions_per_trial=2, # the number of models generated on each trial
#     overwrite=True, # whether to overwrite previous trials
#     directory="hyperparams/RandomSearch", # the directory to save the trials
#     project_name="1", # the name of the project
# )  

# # get a summary of the range of hyperparameters to tune


random_tuner = customTuner(input_shape, output_shape, 
                            objective='val_loss', 
                            max_trials=3,
                            executions_per_trial=1,
                            overwrite=True,
                            directory='hyperparams/RandomSearch',
                            project_name='Black-Scholes')
random_tuner.search_space_summary()

Search space summary
Default search space size: 0


In [11]:
# start the search
random_tuner.search(train_ds, valid_ds, epochs=5)

Trial 3 Complete [00h 01m 21s]
val_loss: 43.519309997558594

Best val_loss So Far: 35.3779296875
Total elapsed time: 00h 04m 50s
INFO:tensorflow:Oracle triggered exit


Get the best compiled models

In [12]:
hypermodel = customHyperModel()

best_model = hypermodel.build(random_tuner.get_best_hyperparameters()[0])

best_model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 5)]               0         
                                                                 
 flatten_4 (Flatten)         (None, 5)                 0         
                                                                 
 dense_12 (Dense)            (None, 200)               1200      
                                                                 
 dense_13 (Dense)            (None, 50)                10050     
                                                                 
 dense_14 (Dense)            (None, 50)                2550      
                                                                 
 dense_15 (Dense)            (None, 1)                 51        
                                                                 
Total params: 13,851
Trainable params: 13,851
Non-trainable

We can view the results of the search and use the hyperparameters displayed to train the model.

In [13]:
random_tuner.results_summary()

Results summary
Results in hyperparams/RandomSearch/Black-Scholes
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 3
learning_rate: 0.0002157147399339997
rate_decay: 0.9794184370846655
l1_coeff: 3.0867496112040877e-07
l2_coeff: 1.276252438847092e-07
units_1: 200
units_2: 50
units_3: 50
Score: 35.3779296875
Trial summary
Hyperparameters:
num_layers: 1
learning_rate: 0.0019513235178244249
rate_decay: 0.958098743823176
l1_coeff: 3.0165387229242814e-07
l2_coeff: 1.1356922506654835e-07
units_1: 250
units_2: 50
units_3: 200
Score: 43.519309997558594
Trial summary
Hyperparameters:
num_layers: 1
learning_rate: 0.0001
rate_decay: 0.85
l1_coeff: 1e-08
l2_coeff: 1e-08
units_1: 50
Score: 56.22092056274414


Now train the best model obtained

In [14]:
best_model.fit(train_ds, epochs = 10, validation_data = valid_ds)

Epoch 1/10
   7/2688 [..............................] - ETA: 25s - loss: 2427.5752 - mean_squared_error: 27.1018  

2022-09-09 19:21:55.573373: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2687/2688 [============================>.] - ETA: 0s - loss: 68.5073 - mean_squared_error: 0.1125

2022-09-09 19:22:19.970756: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2688/2688 [==============================] - 26s 10ms/step - loss: 68.5046 - mean_squared_error: 0.1125 - val_loss: 50.7631 - val_mean_squared_error: 0.0141
Epoch 2/10
2688/2688 [==============================] - 26s 10ms/step - loss: 51.0126 - mean_squared_error: 0.0144 - val_loss: 43.5753 - val_mean_squared_error: 0.0102
Epoch 3/10
2688/2688 [==============================] - 26s 10ms/step - loss: 46.1343 - mean_squared_error: 0.0123 - val_loss: 39.0990 - val_mean_squared_error: 0.0113
Epoch 4/10
2688/2688 [==============================] - 26s 10ms/step - loss: 42.5243 - mean_squared_error: 0.0103 - val_loss: 41.9373 - val_mean_squared_error: 0.0109
Epoch 5/10
2688/2688 [==============================] - 26s 10ms/step - loss: 40.1001 - mean_squared_error: 0.0088 - val_loss: 35.3779 - val_mean_squared_error: 0.0073
Epoch 6/10
2688/2688 [==============================] - 26s 10ms/step - loss: 37.9027 - mean_squared_error: 0.0076 - val_loss: 34.0149 - val_mean_squared_error: 0.0073
Epo

Save the trained model

In [17]:
path_to_model = os.path.join(parent_path, 'NeuralNetwork/models/BS-0909')

In [15]:
best_model.save(path_to_model)

INFO:tensorflow:Assets written to: /Users/customer/projects/UROP-2022/NeuralNetwork/models/BS-0909/assets


To load the model, use `tf.keras.models.load_model`

In [18]:
from tensorflow import keras
model = keras.models.load_model(path_to_model)
model.evaluate(test_ds)

 20/336 [>.............................] - ETA: 1s - loss: 26.0230 - mean_squared_error: 0.0037

2022-09-09 19:37:46.091810: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


336/336 [==============================] - 2s 5ms/step - loss: 27.2961 - mean_squared_error: 0.0044


[27.29612159729004, 0.004369616508483887]

## Some other Tuners to try out  

But this is not the goal of the project right now.

We also try `Hyperband` which generates a large number of models and discard them in a tournament style.

In [ ]:
band_tuner = keras_tuner.Hyperband(
    hypermodel=tuneLR,
    objective="val_loss",
    max_epochs=10,
    factor=3,
    directory="hyperparams/Hyperband",
    project_name="1"
)
band_tuner.search_space_summary()
band_tuner.search(train_ds, epochs = 2, validation_data = valid_ds)  

In [ ]:
band_tuner.results_summary()

Finally, we try `BayesianOptimization`.

In [ ]:
bayes_tuner = keras_tuner.BayesianOptimization(
    hypermodel=tuneLR,
    objective="val_loss",
    max_trials=3,
    directory="hyperparams/BayesianOptimization",
    project_name="1"
)
bayes_tuner.search_space_summary()
bayes_tuner.search(train_ds, epochs = 2, validation_data = valid_ds) 

In [ ]:
bayes_tuner.results_summary()